In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seed-iv/Channel Order.xlsx
/kaggle/input/seed-iv/ReadMe.txt
/kaggle/input/seed-iv/eye_feature_smooth/2/4_20151118.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/14_20151208.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/5_20160413.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/3_20151018.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/6_20150511.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/15_20150514.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/10_20151021.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/12_20150804.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/13_20151125.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/11_20150921.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/8_20151110.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/9_20151119.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/2_20150920.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/7_20150717.mat
/kaggle/input/seed-iv/eye_feature_smooth/2/1_20161125.mat
/kaggle/input/seed-iv/eye_feature_smooth/3/14_2015

In [2]:
import numpy as np
import tensorflow as tf

import scipy
import os
from os import listdir
from scipy.io import loadmat
import pickle
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from tensorflow.keras import regularizers
from tensorflow.keras import activations

In [3]:
import os
import numpy as np
from scipy.io import loadmat

# Correct paths for Kaggle dataset
directories = ["/kaggle/input/seed-iv/eeg_feature_smooth/{}".format(i+1) for i in range(3)] 
print(directories)

channel_coords = [
    ['0', '0', 'AF3', 'FP1', 'FPZ', 'FP2', 'AF4', '0', '0'], 
    ['F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8'], 
    ['FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8'], 
    ['T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8'], 
    ['TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8'], 
    ['P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8'], 
    ['0', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', '0'], 
    ['0', '0', 'CB1', 'O1', 'OZ', 'O2', 'CB2', '0', '0']
]

channel_list = [
    'FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 
    'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 
    'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 
    'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 
    'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 
    'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 
    'O1', 'OZ', 'O2', 'CB2'
]

# Verifying the length of channel coordinates
print(len(channel_coords), len(channel_coords[0]))

# Creating a dictionary to map channel names to coordinates
coord_dict = {}
for n in range(len(channel_list)):
    for i, l in enumerate(channel_coords):
        for j, x in enumerate(l):
            if channel_list[n] == x:
                coord_dict[n] = (i, j)
print(coord_dict)

# Initialize the array with the correct shape
n = 24
perSample = ['de_movingAve', 'de_LDS', 'psd_movingAve', 'psd_LDS']
array = np.zeros(shape=(len(directories), len(os.listdir(directories[0])), n, 4, 8, 9, 5, 64)) 

# Iterate over directories to load and process data
for h, dire in enumerate(directories):
    print(dire)
    data = [loadmat(os.path.join(dire, file)) for file in os.listdir(dire)]
    for i, bigsample in enumerate(data):
        for j in range(n):
            for k, key in enumerate(perSample):
                # Transpose and pad the sample
                sample = np.transpose(np.array(bigsample[key + str(j+1)]), (0, 2, 1))
                sample = np.pad(sample, [(0, 0), (0, 0), (0, 64 - sample.shape[2])])
                # Fill the array based on channel locations
                for l, channel in enumerate(sample):
                    array[h][i][j][k][coord_dict[l][0]][coord_dict[l][1]] = channel

# Output the shape of the final array
print(array.shape)


['/kaggle/input/seed-iv/eeg_feature_smooth/1', '/kaggle/input/seed-iv/eeg_feature_smooth/2', '/kaggle/input/seed-iv/eeg_feature_smooth/3']
8 9
{0: (0, 3), 1: (0, 4), 2: (0, 5), 3: (0, 2), 4: (0, 6), 5: (1, 0), 6: (1, 1), 7: (1, 2), 8: (1, 3), 9: (1, 4), 10: (1, 5), 11: (1, 6), 12: (1, 7), 13: (1, 8), 14: (2, 0), 15: (2, 1), 16: (2, 2), 17: (2, 3), 18: (2, 4), 19: (2, 5), 20: (2, 6), 21: (2, 7), 22: (2, 8), 23: (3, 0), 24: (3, 1), 25: (3, 2), 26: (3, 3), 27: (3, 4), 28: (3, 5), 29: (3, 6), 30: (3, 7), 31: (3, 8), 32: (4, 0), 33: (4, 1), 34: (4, 2), 35: (4, 3), 36: (4, 4), 37: (4, 5), 38: (4, 6), 39: (4, 7), 40: (4, 8), 41: (5, 0), 42: (5, 1), 43: (5, 2), 44: (5, 3), 45: (5, 4), 46: (5, 5), 47: (5, 6), 48: (5, 7), 49: (5, 8), 50: (6, 1), 51: (6, 2), 52: (6, 3), 53: (6, 4), 54: (6, 5), 55: (6, 6), 56: (6, 7), 57: (7, 2), 58: (7, 3), 59: (7, 4), 60: (7, 5), 61: (7, 6)}
/kaggle/input/seed-iv/eeg_feature_smooth/1
/kaggle/input/seed-iv/eeg_feature_smooth/2
/kaggle/input/seed-iv/eeg_feature_sm

In [4]:
_X = array.reshape(np.prod(array.shape[0:3]), *array.shape[3:])
print(_X.shape)
X_loso = array[:,:,:,1,:,:,]
X_loso = np.transpose(X_loso, (0,1,2,6,3,4,5))
print(X_loso.shape)

(1080, 4, 8, 9, 5, 64)
(3, 15, 24, 64, 8, 9, 5)


In [5]:
session1_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3]
session2_label = [2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1]
session3_label = [1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
labels = {0: 'neutral', 1: 'sad', 2: 'fear', 3: 'happy'}

y = np.array(session1_label * 15 + session2_label * 15 + session3_label * 15)

print(y.shape)
y_loso = np.reshape(y, (3,15,24))
print(y_loso.shape)

(1080,)
(3, 15, 24)


In [7]:
def crossval(generate_model, n_epochs, X_train, y_train, X_test, y_test, filename = None):
    kfold = StratifiedKFold(n_splits=4, shuffle=True, random_state=0)
    bestmodel = None
    bestAcc = 0
    cvscores = []
    fold = 1
    for train, test in kfold.split(X_train, y_train):
        model = generate_model()
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold} ...')
        model.fit(X_train[train], y_train[train],epochs=n_epochs, verbose=1) # validation_split=0.2)
        scores = model.evaluate(X_train[test], y_train[test], verbose=1)
        print("Score for fold %d - %s: %.2f%%" % (fold, model.metrics_names[1], scores[1]*100))
        if(scores[1] > bestAcc):
            bestAcc = scores[1]
            bestmodel = model
        cvscores.append(scores[1] * 100)
        fold += 1
    print('------------------------------------------------------------------------')
    print("Avg accuracies: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    test_loss, test_acc = bestmodel.evaluate(X_test,  y_test, verbose=2)
    print('\nTest accuracy:', test_acc)
    if filename:
        pickle.dump( bestmodel, open( filename, "wb" ) )

In [10]:
def twodConv():
    return tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu",input_shape=X.shape[1:]),
    tf.keras.layers.MaxPooling2D(
    pool_size=2, strides=2, data_format="channels_last"),
    tf.keras.layers.BatchNormalization(center=True, scale=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

In [24]:
X = _X.transpose(0, 5, 1,2,3,4)
print(X.shape)
X = X.reshape(X.shape[0], X.shape[1], np.prod(X.shape[2:]))
print(X.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 4, 8, 9, 5)
(1080, 64, 1440)


In [51]:
X = _X.transpose(0, 5, 2,3, 4, 1)[:,:,:,:,:,1]

X = X.reshape(X.shape[0], X.shape[1], X.shape[2], X.shape[3], np.prod(X.shape[4:]))
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

(1080, 64, 8, 9, 5)


In [64]:
def convlstm2d():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(32, kernel_size=3, padding="same", return_sequences = False, input_shape=(64, 8, 9, 5)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(4))
    return model

In [61]:
import tensorflow as tf

def transformer_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=16, input_shape=(64, 8, 9, 5)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(4))
    return model


In [67]:
crossval(convlstm2d,50, X_train , y_train , X_test , y_test)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 158ms/step - accuracy: 0.3580 - loss: 1.3172
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.4578 - loss: 1.1348
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.4939 - loss: 1.0475
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.4879 - loss: 1.0517
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.4768 - loss: 1.0361
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5071 - loss: 1.0045
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5518 - loss: 0.9695
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5586 - loss: 0.9318
Epoch 9/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5512 - loss: 0.9233
Epoch 10/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5852 - loss: 0.9232
Epoch 11/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step